In [1]:
import numpy as np
import pandas as pd

def aggregate_shap_explanations(explanations: list):
    """
    Agrupa contribuciones SHAP por feature a través de todos los folds/samples.

    Parámetros
    ----------
    explanations : list[shap.Explanation]
        Lista devuelta por shap_after_nested_cv(...), una Explanation por fold.
        Cada Explanation puede tener múltiples samples en el test de ese fold.

    Devuelve
    --------
    df_agg : pd.DataFrame
        Columnas:
          - feature
          - count           (# de contribuciones sumadas = muestras × folds donde la feature estuvo)
          - mean_shap       (promedio con signo, solo sobre apariciones)
          - mean_abs_shap   (promedio absoluto, solo sobre apariciones)
        Ordenado por mean_abs_shap descendente.
    top20 : pd.DataFrame
        Subconjunto de df_agg con las 20 principales por mean_abs_shap.
    """
    # Acumuladores por feature
    sum_signed = {}   # suma de contribuciones (con signo)
    sum_abs = {}      # suma de |contribuciones|
    count = {}        # número de contribuciones agregadas (samples donde apareció la feature)

    for expl in explanations:
        # ---- Extraer valores y nombres de features ----
        values = np.array(expl.values)
        feat_names = getattr(expl, "feature_names", None)
        if feat_names is None:
            # Fallback (raro): intentar expl.feature_names, expl.data_feature_names
            feat_names = getattr(expl, "data_feature_names", None)
        if feat_names is None:
            raise ValueError("No se encontraron nombres de features en la Explanation.")

        feat_names = np.asarray(feat_names)
        # ---- Manejo de dimensiones ----
        # Casos típicos:
        #  - (n_samples, n_features)           -> binaria/regresión
        #  - (n_samples, n_classes, n_features)-> multiclase
        #  - (n_features,)                     -> 1 muestra, 1 output
        if values.ndim == 1:
            # -> (1, n_features)
            values = values[None, :]
        if values.ndim == 2:
            # No hay eje de clases
            # shap_signed y shap_abs a nivel sample-feature ya están listos
            shap_signed_sf = values                        # (n_samples, n_features)
            shap_abs_sf = np.abs(values)                   # (n_samples, n_features)
        elif values.ndim == 3:
            # Eje de clases presente. Convención SHAP: (n_samples, n_outputs, n_features)
            # Requisito: colapsar clases promediando por clase (abs y con signo).
            shap_signed_sf = values.mean(axis=1)           # (n_samples, n_features) promedio con signo
            shap_abs_sf = np.abs(values).mean(axis=1)      # (n_samples, n_features) promedio de absolutos
        else:
            raise ValueError(f"Dimensión de valores SHAP no soportada: {values.shape}")

        n_samples, n_features = shap_signed_sf.shape
        if len(feat_names) != n_features:
            raise ValueError(
                f"Longitud de feature_names ({len(feat_names)}) no coincide con n_features ({n_features})."
            )

        # ---- Agregar sobre samples del fold actual ----
        # Suma por-feature sobre samples
        fold_sum_signed = shap_signed_sf.sum(axis=0)  # (n_features,)
        fold_sum_abs = shap_abs_sf.sum(axis=0)        # (n_features,)

        # Cada feature cuenta tantas "apariciones" como número de samples del test de este fold
        fold_counts = np.full(n_features, n_samples, dtype=int)

        # Actualizar acumuladores globales (solo features presentes en este expl)
        for j, fname in enumerate(feat_names):
            sum_signed[fname] = sum_signed.get(fname, 0.0) + float(fold_sum_signed[j])
            sum_abs[fname]    = sum_abs.get(fname, 0.0)    + float(fold_sum_abs[j])
            count[fname]      = count.get(fname, 0)        + int(fold_counts[j])

    # ---- Construir DataFrame final ----
    data = []
    for fname in count.keys():
        c = count[fname]
        # Evitar división por cero (por seguridad; no debería ocurrir)
        mean_signed = sum_signed[fname] / c if c > 0 else np.nan
        mean_abs = sum_abs[fname] / c if c > 0 else np.nan
        data.append((fname, c, mean_signed, mean_abs))

    df_agg = pd.DataFrame(data, columns=["feature", "count", "mean_shap", "mean_abs_shap"])
    df_agg = df_agg.sort_values("mean_abs_shap", ascending=False, kind="mergesort").reset_index(drop=True)

    top20 = df_agg.head(20).copy()
    return df_agg, top20
